# Analisis exploratorio de datos

In [2]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('../data/comentarios_mejorados.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Obtener información sobre el DataFrame
print(df.info())

# Obtener estadísticas descriptivas del DataFrame
print(df.describe())

ModuleNotFoundError: No module named 'pandas'

## 1. Elimitar columnas que no aportan informacion
En este caso se eliminan las columnas que no aportan informacion al modelo y nos quedamos con el comentario.

In [ ]:
columnas_a_eliminar = ['channelId', 'videoId', 'textOriginal', 'authorDisplayName', 
                       'authorProfileImageUrl', 'authorChannelUrl', 'authorChannelId', 
                       'canRate', 'viewerRating', 'likeCount', 'publishedAt', 
                       'updatedAt', 'parentId']

# Eliminar las columnas
df = df.drop(columnas_a_eliminar, axis=1)

# Mostrar las primeras filas del DataFrame
print(df.head())

## 2. Limpieza de texto
Se eliminan los caracteres especiales, se pasa todo a minusculas y se eliminan espaacios en blanco innecesarios.

In [ ]:
import re

# Función para limpiar el texto
def clean_text(text):
    # Eliminar caracteres especiales y emojis
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    # Minúsculas
    text = text.lower()
    # Eliminar espacios en blanco innecesarios
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Aplicar la función de limpieza de texto a la columna 'textDisplay'
df['textDisplay'] = df['textDisplay'].apply(clean_text)

# Mostrar las primeras filas del DataFrame
print(df.head())

## 3. Tokenizacion
Se separa el texto en palabras individuales. Esto nos permite analizar cada palabra por separado y no como una cadena de texto.


In [ ]:
import nltk
# Descargar el paquete 'punkt' de NLTK solo es necesario la primera vez
nltk.download('punkt')

# Tokenizar el texto
df['textDisplay'] = df['textDisplay'].apply(nltk.word_tokenize)

print(df.head())


## 4. Eliminacion de stopwords
Se eliminan las palabras que no aportan informacion al modelo. Estas palabras son las que se encuentran en la lista de stopwords como por ejemplo: "el", "la", "de", etc.

In [ ]:
# Eliminar las palabras vacías
from nltk.corpus import stopwords
# Descargar las palabras vacías de NLTK solo es necesario la primera vez
nltk.download('stopwords')
stopwords = stopwords.words('spanish')

# Función para eliminar las palabras vacías
def remove_stopwords(text):
    text = [word for word in text if word not in stopwords]
    return text

# Eliminar las palabras vacías
df['textDisplay'] = df['textDisplay'].apply(remove_stopwords)

print(df.head())

## 5. Stemming o lematizacion
Se eliminan los sufijos de las palabras para quedarnos con la raiz de la palabra. Esto nos permite reducir el numero de palabras que se tienen que analizar. Por ejemplo: "corriendo" se convierte en "correr". 

Esto es útil en análisis de sentimientos ya que la raíz de la palabra puede ser más importante que la palabra en sí. Por ejemplo, "bueno" y "mejor" tienen la misma raíz, "buen", por lo que se puede decir que son similares.

Para lematizar se utiliza la libreria de Spacy, ya que ntltk no tiene soporte para el idioma español.

En nuestro caso instalaremos una versión precompilada de Spacy para el idioma español, ya que la compilación de la libreria puede tardar mucho tiempo.

" pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-2.2.0/es_core_news_sm-2.2.0.tar.gz "

In [ ]:
import spacy

# Cargar el modelo de lenguaje español de Spacy
nlp = spacy.load('es_core_news_sm')

# Función para lematizar el texto
def lemmatization(texts):
    output = []
    for text in texts:
        doc = nlp(" ".join(text)) 
        output.append([token.lemma_ for token in doc])
    return output

# Lematizar el texto
df['textDisplay'] = lemmatization(df['textDisplay'])

print(df.head())